# Model Training

This notebook demonstrates training the LSTM model for gesture recognition.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import tensorflow as tf
from pathlib import Path

from src.data.loader import GestureDataLoader
from src.models.lstm_model import GestureLSTM
from src.training.trainer import ModelTrainer
from src.training.metrics import calculate_metrics

## Load Training Data

In [ ]:
# Initialize data loader
data_loader = GestureDataLoader(
    data_path=Path('../data/processed/processed_en.json'),
    batch_size=32,
    validation_split=0.2
)

train_dataset, val_dataset = data_loader.get_datasets()
vocab_size = data_loader.vocab_size

print(f"Vocabulary size: {vocab_size}")
print(f"Training batches: {len(list(train_dataset))}")
print(f"Validation batches: {len(list(val_dataset))}")

## Build Model

In [ ]:
# Create model
model = GestureLSTM(
    vocab_size=vocab_size,
    embedding_dim=128,
    lstm_units=256,
    dropout_rate=0.3
)

model.build(input_shape=(None, 50, 3))  # (batch, sequence_length, features)
model.summary()

## Train Model

In [ ]:
# Initialize trainer
trainer = ModelTrainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    checkpoint_dir=Path('../models/checkpoints')
)

# Train
history = trainer.train(
    epochs=50,
    learning_rate=0.001
)

## Evaluate Model

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(history.history['loss'], label='Train Loss')
ax1.plot(history.history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history.history['accuracy'], label='Train Accuracy')
ax2.plot(history.history['val_accuracy'], label='Val Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Test Predictions

In [ ]:
# Test on sample data
from src.inference.predictor import GesturePredictor

predictor = GesturePredictor(
    model_path=Path('../models/checkpoints/best_model.h5'),
    vocab_path=Path('../data/processed/vocab.json')
)

# Load test sample
test_sample = next(iter(val_dataset))
predictions = predictor.predict_batch(test_sample[0][:5])

print("Sample predictions:")
for i, pred in enumerate(predictions):
    print(f"{i+1}. {pred['word']} (confidence: {pred['confidence']:.2f})")